In [334]:
# Imports

import sagemaker
from sagemaker import image_uris
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.pytorch import PyTorch
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.lambda_step import LambdaStep
from sagemaker.lambda_helper import Lambda
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.workflow.functions import Join
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig
from sagemaker.workflow.monitor_batch_transform_step import MonitorBatchTransformStep
from sagemaker.workflow.steps import CreateModelStep



In [335]:
# General Pipeline Information

session = sagemaker.Session()
sagemaker_role = sagemaker.get_execution_role()

pipeline_name = "cs401FinalBirdclefPipeline"
region = "us-east-1"

In [336]:
# Step 1: Pre-Processing. Turns .ogg files into Spectograms

spectro_processor = ScriptProcessor(
    role=sagemaker_role,
    image_uri=image_uris.retrieve("pytorch", region, "2.0.0", "py310", instance_type="ml.t3.medium", image_scope="training"),
    command=['python3'],
    instance_count=1,
    instance_type='ml.t3.medium',
    base_job_name='audio-processing',
    sagemaker_session=session
)

step_spectro = ProcessingStep(
    name="CreateSpectrograms",
    processor=spectro_processor,
    code="pipeline/process_audio.py"
)


In [337]:
# Step 2: Make manifests. Manifests link every input file to a class / label so the model can train

manifest_processor = ScriptProcessor(
    image_uri=image_uris.retrieve("pytorch", region, "2.0.0", "py310", instance_type="ml.t3.medium", image_scope="inference"),
    command=["python3"],
    role=sagemaker_role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="build-manifests",
    sagemaker_session=session,
)

step_manifest = ProcessingStep(
    name="BuildManifests",
    processor=manifest_processor,
    code="pipeline/manifest_build.py",
    job_arguments=[
        "--specs-s3-prefix", "s3://cs401finalpipelineprocessingdata/data/audio_specs/",
        "--train-csv-s3-uri", "/opt/ml/processing/input/train/train.csv",
        "--output-dir", "/opt/ml/processing/output"
    ],
    inputs=[
        ProcessingInput(
            source="s3://cs401finalpipelineprocessingdata/data/audio_specs/",
            destination="/opt/ml/processing/input/specs",
            input_name="SPECS",
        ),
        ProcessingInput(
            source="s3://cs401finalpipelineinput/train.csv",
            destination="/opt/ml/processing/input/train",
            input_name="TRAINCSV",
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="Manifests",
            source="/opt/ml/processing/output",
        ),
    ],
)

In [338]:
# Step 3: Train the model. Train the efficient net model using the spectograms and manifests

pytorch_estimator = PyTorch(
    entry_point="pipeline/train_script_2.0.py",
    source_dir=".",
    role=sagemaker_role,
    framework_version="2.6.0",
    py_version="py312",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    hyperparameters={
        "epochs": 20,
        "batch-size": 32,
        "learning-rate": 0.001,
        "accumulation-steps": 4,
        "train-manifest": "/opt/ml/input/data/manifests/train_manifest.csv",
        "val-manifest": "/opt/ml/input/data/manifests/val_manifest.csv",
    },
    dependencies=['pipeline/model_dependencies/requirements.txt']
)

step_train = TrainingStep(
    name="TrainSpectrogramModel",
    estimator=pytorch_estimator,
    inputs={
        "specs": "s3://cs401finalpipelineprocessingdata/data/audio_specs/",
        "manifests": step_manifest.properties.ProcessingOutputConfig
                                  .Outputs["Manifests"]
                                  .S3Output.S3Uri,
    },
)

In [339]:
# Step 4: Package the Model. Zip the model into a tar with the model_dependencies folder (inference.py and requirements.txt)

packaging_processor = ScriptProcessor(
    image_uri=image_uris.retrieve("pytorch", region, "2.6.0", "py312", instance_type="ml.t3.medium", image_scope="inference"),
    role=sagemaker_role,
    instance_type="ml.t3.medium",
    instance_count=1,
    command=["python3"]
)

step_package = ProcessingStep(
    name="PackageModel",
    processor=packaging_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source="pipeline/model_dependencies/",
            destination="/opt/ml/processing/input/dependencies",
            input_name="model_code"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="PackagedModel",
            source="/opt/ml/processing/output"
        )
    ],
    code="pipeline/package_model.py"
)

In [340]:
# Step 5: Create the Pytorch Model

s3_uri = Join(on="/", values=[step_package.properties.ProcessingOutputConfig.Outputs["PackagedModel"].S3Output.S3Uri, "model.tar.gz"])

pytorch_model = PyTorchModel(
    model_data=s3_uri,
    role=sagemaker_role,
    framework_version="2.6.0",
    py_version="py312",
    sagemaker_session=session,
)


step_create_model = CreateModelStep(
    name="CreateBirdCLEFModelResource",
    model=pytorch_model,
    inputs=sagemaker.inputs.CreateModelInput(instance_type="ml.m5.large")
)

In [341]:
# Step 6: Register the model.


step_register = RegisterModel(
    name="RegisterBirdclefModel",
    model=pytorch_model,
    model_package_group_name="BirdclefModelPackageGroup",
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    approval_status="Approved",
)

In [342]:
# Step 7: Deploy the Model so it can be used in production.

lambda_function_name = "sagemaker-deploy-model-lambda"

deploy_model_lambda_function = Lambda(
    function_name=lambda_function_name,
    execution_role_arn=sagemaker_role,
    script="pipeline/lambda_endpoint_deployer.py",
    handler="lambda_endpoint_deployer.lambda_handler",
)

lambda_step_inputs = {
    "endpoint_name": "birdclef-endpoint",
    "model_name": step_create_model.properties.ModelName,
    "instance_type": "ml.m5.large",
}

lambda_step = LambdaStep(
    name="DeployModelLambdaStep",
    lambda_func=deploy_model_lambda_function,
    inputs=lambda_step_inputs,
)


In [343]:
# Create the Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[],
    steps=[step_spectro, step_manifest, step_train, step_package, step_create_model, step_register, lambda_step],
    sagemaker_session=session
)
pipeline.upsert(role_arn=sagemaker_role)

[04/24/25 03:57:26] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=914057;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=625143;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=322746;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=404713;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[04/24/25 03:57:50] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=134572;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=388187;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=167271;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=317064;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

[04/24/25 03:57:51] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=680900;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=237704;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

...

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=808046;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=426472;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=907650;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=46376;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=662435;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/_utils.py\_utils.py]8;;\:]8;id=847606;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=812848;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=39390;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:084375543672:pipeline/cs401FinalBirdclefPipeline',
 'ResponseMetadata': {'RequestId': '30d9597d-36ae-4207-9b6e-4d5bcff3b617',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '30d9597d-36ae-4207-9b6e-4d5bcff3b617',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Thu, 24 Apr 2025 03:58:18 GMT'},
  'RetryAttempts': 0}}

In [344]:
# Run the Pipeline

pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:084375543672:pipeline/cs401FinalBirdclefPipeline/execution/v6lpqxg4k60s', sagemaker_session=<sagemaker.session.Session object at 0x7f784f905e20>)